In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import pylab as pl
import pickle
from sklearnex import patch_sklearn
patch_sklearn()
from sklearn import svm
from sklearn.tree import DecisionTreeClassifier 
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.model_selection import cross_val_score
from sklearn import metrics

Intel(R) Extension for Scikit-learn* enabled (https://github.com/intel/scikit-learn-intelex)
2022-04-11 21:17:34.702960: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-04-11 21:17:34.702992: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


### Libraries

| Library      | Description |
| ------------ | ----------- |
| numpy        | for array computation |
| matplotlib   | Python plotting package |
| pylab        | utilities |
| sklearnex    | Intel(R) Extension for Scikit-learn is a seamless way to speed up your Scikit-learn application |
| sklearn      | A set of python modules for machine learning and data mining |
| pickle       | Python object serialization |


### The dataset

The dataset we used for this project is the MNIST ("Modified National Institute of Standards and Technology") of handwritten digits, it contains 70000 sample of 784 grayscale pixel of the 10 digits labeled from (0-9)

In [2]:
df = pd.read_csv("mnist_dataset.csv")
df.head()

,label,1x1,1x2,1x3,1x4,1x5,1x6,1x7,1x8,1x9,...,28x19,28x20,28x21,28x22,28x23,28x24,28x25,28x26,28x27,28x28
0,5,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,4,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,9,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [3]:
x = df.drop('label', axis='columns')
y = df['label']

In [ ]:
x.head()

In [ ]:
y.head()

In [14]:
x.shape, y.shape

((70000, 784), (70000,))

In [4]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.3, random_state=29)

In [5]:
x_train.shape, y_train.shape, x_test.shape, y_test.shape

((49000, 784), (49000,), (21000, 784), (21000,))

# tree

In [ ]:
tree_clf = DecisionTreeClassifier(criterion="entropy", random_state=0)
tree_clf = tree_clf.fit(x_train, y_train)

y_predict_train = tree_clf.predict(x_train)
y_predict_test = tree_clf.predict(x_test)

print(f'Accuracy of the training sample = {accuracy_score(y_train, y_predict_train)*100}%')
print(f'Accuracy of the test sample = {round(accuracy_score(y_test, y_predict_test), 2)*100}%')

print("Tree size:", tree_clf.tree_.node_count)


In [ ]:
path = tree_clf.cost_complexity_pruning_path(x_train, y_train)
alphas = path['ccp_alphas']

new_alphas = []

temp = 1
for alpha in alphas:
    if abs(alpha - temp)> 0.0001:
        new_alphas.append(alpha)
        temp = alpha

print(len(alphas))
print(len(new_alphas))

In [ ]:
acc_train = []
acc_test = []
tree_size = []
tree_depth = []
alpha_values = []
temp = -1

for alpha in new_alphas:
    if alpha != temp:
        temp = alpha
        temp_tree = DecisionTreeClassifier(ccp_alpha=alpha, criterion="entropy", random_state=0)
        temp_tree.fit(x_train, y_train)

        y_predect_train, y_predect_test = temp_tree.predict(x_train), temp_tree.predict(x_test)
        
        scores = cross_val_score(temp_tree, x_train, y_train, cv=5)
        
        alpha_values.append([alpha, np.mean(scores), np.std(scores)])
        tree_depth.append(temp_tree.get_depth())
        tree_size.append(temp_tree.tree_.node_count)
        acc_train.append(accuracy_score(y_train, y_predect_train))
        acc_test.append(accuracy_score(y_test, y_predect_test))
        print(alpha)

In [ ]:
plt.figure(figsize=(14, 7))
plt.plot(tree_size, acc_train, label='train_accuracy')
plt.plot(tree_size, acc_test, label='test_accuracy')
plt.legend()
plt.xlabel("size(tree)")
plt.ylabel("Accuracy")
plt.title('Accuracy vs tree(size) (Figure 1)')
plt.show()

In [ ]:
plt.figure(figsize=(14, 7))
plt.plot(new_alphas, tree_depth)
plt.xticks(ticks=np.arange(0.00, 0.26, 0.005), rotation=60)
plt.xlabel("alphas")
plt.ylabel("tree depth")
plt.title('depth vs alphas (Figure 2)')
plt.grid()
plt.show()

In [ ]:
plt.figure(figsize=(35, 14))
plt.scatter(new_alphas, acc_train)
plt.scatter(new_alphas, acc_test)
plt.plot(new_alphas, acc_train, label='train_accuracy', drawstyle="steps-post")
plt.plot(new_alphas, acc_test, label='test_accuracy', drawstyle="steps-post")
plt.xlabel("alphas")
plt.ylabel("accuracy")
plt.xticks(ticks=np.arange(0.00, 0.2, 0.005), rotation=60)
plt.yticks(ticks=np.arange(0.00, 1.00, 0.05))
plt.legend()
plt.title('Accuracy vs alpha (Figure 3)')
plt.grid()
plt.show()

In [ ]:
tree_clf = DecisionTreeClassifier(random_state=0, criterion='entropy', ccp_alpha=0.00010134290211517562
)
tree_clf = tree_clf.fit(x_train, y_train)

y_predict_test1 = tree_clf.predict(x_test)
y_predict_train1 = tree_clf.predict(x_train)

print(f'Accuracy of the training sample = {round(accuracy_score(y_train, y_predict_train1), 3)*100}%')
print(f'Accuracy of the test sample = {round(accuracy_score(y_test, y_predict_test1), 3)*100}%')

# KNN

# SVM

### Linear

In [6]:
#model_lin = svm.SVC(kernel='linear', degree=3, gamma='scale')
#model_lin.fit(x_train, y_train)

In [ ]:
#pickle.dump(model_lin, open('linear_SVM_model.sav', 'wb'))

In [7]:
model_lin_ser = pickle.load(open('linear_SVM_model.sav', 'rb'))
print(model_lin_ser)


SVC(kernel='linear')


In [8]:
y_pred_train = model_lin_ser.predict(x_train)
y_pred_test = model_lin_ser.predict(x_test)

In [9]:
print(accuracy_score(y_train, y_pred_train))
print(accuracy_score(y_test, y_pred_test))

0.9896122448979592
0.916952380952381


### rbf

In [10]:
model_rbf = svm.SVC(kernel='rbf', degree=3, gamma='scale')
model_rbf.fit(x_train, y_train)

SVC()

In [11]:
y_pred_train = model_rbf.predict(x_train)
y_pred_test = model_rbf.predict(x_test)

In [12]:
print(accuracy_score(y_train, y_pred_train))
print(accuracy_score(y_test, y_pred_test))

0.9891020408163266
0.9801428571428571


In [19]:
kf = cross_val_score(model_rbf, x, y, cv=10)
kf

array([0.98128571, 0.97985714, 0.979     , 0.97714286, 0.97785714,
       0.97757143, 0.97385714, 0.97771429, 0.979     , 0.98442857])

In [20]:
print(type(kf))
np.average(kf)

<class 'numpy.ndarray'>


0.9787714285714285

In [ ]:
plt.imshow(x_train[2], cmap= "binary")
plt.title(y_train[2])

In [ ]:
p = np.array([0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,2,4,4,4,3,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,54,112,126,131,85,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2,0,57,214,246,219,204,209,247,180,7,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,208,182,37,0,0,0,36,254,74,0,3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,228,84,0,4,1,2,0,202,133,0,4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2,0,191,167,0,7,2,3,2,191,144,0,4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,3,0,68,255,55,0,0,0,0,218,119,0,4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2,0,149,249,109,40,0,59,251,44,3,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,2,0,126,228,244,199,241,183,0,0,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,61,252,229,233,194,76,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2,5,7,187,205,15,49,165,253,147,0,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2,0,166,232,22,0,0,0,56,255,105,0,3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,3,0,54,255,49,0,4,2,6,0,121,222,2,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,4,0,151,196,0,4,1,2,7,0,87,238,6,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,2,0,191,142,0,4,0,0,0,42,236,147,0,3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,215,137,2,0,9,59,156,250,169,8,2,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2,0,150,247,225,218,233,246,198,84,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,3,54,93,108,85,33,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2,3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,2,3,4,3,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0])
p1 = p.reshape(-1,784)
print(model_lin.predict(p1))